# VAE Training on CIFAR-10
**Variational Autoencoder**

TODO: Partner's implementation

## 1. Setup

In [ ]:
!git clone https://github.com/YOUR_USERNAME/ML2_final.git
%cd ML2_final

In [ ]:
!pip install wandb tqdm -q

In [ ]:
import torch
import torch.optim as optim
import os
from tqdm.auto import tqdm

# TODO: Implement VAE in src/models/vae.py first
from src.models import VAE
from src.losses import vae_loss
from src.data import get_dataloader, denormalize
from src.utils import show_samples, save_samples

## 2. Configuration

In [ ]:
config = {
    'latent_dim': 128,
    'beta': 1.0,  # KL weight (beta-VAE)
    
    'epochs': 200,
    'batch_size': 128,
    'lr': 1e-4,
    
    'sample_every': 10,
    'save_every': 25,
    
    'seed': 42
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 3. Initialize

In [ ]:
torch.manual_seed(config['seed'])
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('samples', exist_ok=True)

train_loader = get_dataloader(batch_size=config['batch_size'])

# TODO: Initialize model after implementing VAE
model = VAE(latent_dim=config['latent_dim']).to(device)
optimizer = optim.Adam(model.parameters(), lr=config['lr'])

## 4. Training

TODO: Implement training loop

In [ ]:
# TODO: Implement training loop
# for epoch in range(config['epochs']):
#     for images, _ in train_loader:
#         images = images.to(device)
#         
#         x_recon, mu, log_var = model(images)
#         loss, recon_loss, kl_loss = vae_loss(images, x_recon, mu, log_var, config['beta'])
#         
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#
#     # Generate samples
#     if (epoch + 1) % config['sample_every'] == 0:
#         samples = model.sample(64, device)
#         show_samples(samples)

## 5. Evaluation

TODO: Add FID calculation, reconstruction visualization